In [6]:
from scipy.integrate import solve_ivp
from scipy.optimize import root, minimize
import numpy as np
import matplotlib.pyplot as plt
import time



def Simulator_v3(td, tm, activity, test=False, activities=[100, 100], tss=[0, 0], final_values=False):
    '''
    This function, Simulator, calculates the number of the atoms at the measurement time, tm, for 
    the case that the person has died at time td. The unit of the time values is day.
    This function solves a set if differential equations to find the number if the atoms.
    '''
    
    decay_time = td
    time_of_measurement = tm
    
    
    lds = [0.18, 0.000085, 0.139, 0.005007]
    Activity = activity

    #--This is for calculation of # of atoms for lead over time e.g. a month
    if(test):
        N0_Rn = 0
    else:
        N0_Rn = (int(Activity) * 3.8 * 24 * 3600)/0.693
    N0_Pb = 0
    N0_Bi = (lds[1]/lds[2])*N0_Pb
    N0_Po = (lds[1]/lds[3])*N0_Pb
    
    tss = np.append(tss, td)
    def acts(t):
        for i in range(1, len(tss)):
            if(t<tss[i])and(t>=tss[i-1]):
                return activities[i-1]
        return activities[-1]
    
    
    # Funtion f(t, N) and is the function that describes the differential equation:
    # f(t, N) = dN/dt where N is an array the number of atoms for different compounds.
    def f(t, N):
        if(test):
            act = acts(t) if t<=decay_time else 0
            #N1decay = 0 if t<decay_time else lds[0]
            return [act-lds[0]*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]
        else:
            N1decay = 0 if t<decay_time else lds[0]
            return [-N1decay*N[0],
                    lds[0]*N[0]-lds[1]*N[1],
                    lds[1]*N[1]-lds[2]*N[2],
                    lds[2]*N[2]-lds[3]*N[3]]

    
    ts = 100
    n = 500
    # C1 and C2 are the number of atoms that we measure in reality.
    time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
    
    # solve_ivp solves the set of differential equations.
    #output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12)
    output = solve_ivp(f, [0, tm*1.1], [N0_Rn, N0_Pb, N0_Bi, N0_Po], rtol=1e-12, t_eval=time)
    
    '''
    if final_values:
        C1 = np.interp(tm, output.t, output.y[1])
        C2 = np.interp(tm, output.t, output.y[2])
        C3 = np.interp(tm, output.t, output.y[3])
        R1 = C1/C3
        R2 = C1/C2
        return tm, R1, R2
    else:
        #ts = 2000
        #n = 500
        # C1 and C2 are the number of atoms that we measure in reality.
        #time = np.concatenate((np.linspace(ts, td, n)[:-1], np.linspace(td, tm, n)))
        #time = np.linspace(ts, tm, n)
        C1 = np.interp(time, output.t, output.y[1])
        C2 = np.interp(time, output.t, output.y[2])
        C3 = np.interp(time, output.t, output.y[3])
        #C1 = output.y[1]
        #C2 = output.y[2]
        #C3 = output.y[3]
        R1 = C1/C3
        R2 = C1/C2
        return time, R1, R2
        #return output.t, R1, R2
    '''
    C1 = output.y[1]
    C2 = output.y[2]
    C3 = output.y[3]
    return output.t, C1/C3, C1/C2


def inverseProblemSolver3(R1t, R2t):   
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.02, 0.01]
    def objective(inp, depth=0):
        td, ms, jump, tc = inp
        if(ms<=0)or(td<=1)or(tc<0)or(tc>tcs[depth])or(jump<=0.1)or(jump>1.8):
            return np.inf
        Rd_act = 100
        _, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        #_, r1, r2 = Simulator_v3(td, td+ms, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        #print(inp)
        return (((R1t-r1[-1])/R1t)**2 + 10*((R2t-r2[-1])/R2t)**2)**0.5
    
    for depth in range(len(tcs)):
        print(depth)
        #solution = minimize(objective, x0=[15000, 10, 0.8, 0.99*tcs[depth]], method='Nelder-Mead', tol=1e-9, args=depth)
        solution = minimize(objective, x0=[15000, 10, 0.4, 0.99*tcs[depth]], method='Nelder-Mead', args=depth,
                            options={'fatol': 1e-9, 'xatol': 1e-4})
        if(solution.success): 
            break
    print('depth: {}'.format(depth))
    return solution


def inverseProblemSolver3_DoubleMeasurement(R1t1, R2t1, R1t2, R2t2, step):
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.02, 0.01]
    def objective(inp, depth=0):
        td, tm, jump, tc = inp
        if(tm<=0)or(td<=1)or(tc<0)or(tc>tcs[depth])or(jump<=0.1)or(jump>1.8):
            return np.inf
        Rd_act = 100
        t, r12, r22 = Simulator_v3(td, td+tm+step, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        r11 = np.interp(td+tm, t, r12)
        r21 = np.interp(td+tm, t, r22)
        #_, r1, r2 = Simulator_v3(td, td+tm, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        #print(inp)
        return (((R1t1-r11)/R1t1)**2 + ((R1t2-r12[-1])/R1t2)**2 + 20*(((R2t1-r21)/R2t1)**2 + ((R2t2-r22[-1])/R2t2)**2))**0.5
    
    for depth in range(len(tcs)):
        print(depth)
        #solution = minimize(objective, x0=[15000, 10, 0.8, 0.99*tcs[depth]], method='Nelder-Mead', tol=1e-9, args=depth)
        solution = minimize(objective, x0=[15000, 10, 0.4, 0.99*tcs[depth]], method='Nelder-Mead', args=depth,
                            options={'fatol': 1e-9, 'xatol': 1e-6})
        if(solution.success)and(abs(solution.fun)<1e-7):
            print(solution.fun)
            break
        else:
            print(solution.fun)
    print('depth: {}'.format(depth))
    return solution


def inverseProblemSolver4(R1t1, R1t2, R2t1, R2t2, step):
    tcs = [0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999]
    #tcs = [0.5, 0.1, 0.01, 0.001]
    tm_guess = np.arange(3, 22, 3)
    def objective(inp, i=0, j=0):
        td, tm, jump, tc = inp
        out_of_bound = (tm<=(0 if i==0 else tm_guess[i-1]))or(tm>tm_guess[i])or(td<=1)or(tc<=0)or(tc>tcs[j])or(jump<=0.1)or(jump>1.8)
        if out_of_bound:
            return np.inf
        Rd_act = 100
        t, r1, r2 = Simulator_v3(td, td+tm+step, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*(1-tc)])
        #_, r1, r2 = Simulator_v3(td, td+tm, Rd_act, True, [Rd_act, Rd_act*jump], [0, td*tc])
        r11, r12 = np.interp([td+tm, td+tm+step], t, r1)
        r21, r22 = np.interp([td+tm, td+tm+step], t, r2)
        return (((R1t1-r11)/R1t1)**2 + ((R1t2-r12)/R1t2)**2 + 10*(((R2t1-r21)/R2t1)**2 + ((R2t2-r22)/R2t2)**2))**0.5
    
    min_sol = None
    min_fun = np.inf
    solved = False
    for j in range(len(tcs)):
        for i in range(len(tm_guess)):
            print("initial guesses: {}\t{}".format(tm_guess[i], tcs[j]))
            solution = minimize(objective, x0=[15000, tm_guess[i], 0.4, 0.999*tcs[j]], method='Nelder-Mead', args=(i, j),
                                options={'fatol': 1e-9, 'xatol': 1e-4})
            if(solution.success):
                if(abs(solution.fun)<min_fun):
                    min_sol = solution
                if(abs(solution.fun)>1e-9):
                    print("Function magnitude: " + str(abs(solution.fun)) + "\ttm: " + str(solution.x[1]))
                    print("Resetting the initial guess...")
                else:
                    solved = True
                    break
        if(solved):
            break
    if not solved:
        solutoin = min_sol
    return solution

In [7]:
files = ['Data_report_no_error_v2_2023_12_18_19_23_53.npy']

data = []

for file in files:
    subdata = np.load(file, allow_pickle=True)
    for j in range(len(subdata)):
        data.append(subdata[j])


inds = []
for i in range(len(data)):
    if abs(data[i]['tm']-data[i]['sol'].x[1])>3:
        inds.append(i)


print('{}/{}'.format(len(inds), len(data)))

5/1000


In [11]:
i = 4

step = 3


%matplotlib qt

print(data[inds[i]])
td = data[inds[i]]['td']
tm = data[inds[i]]['tm']
Rd = data[inds[i]]['Rd']
Tjumps = data[inds[i]]['Tjumps']
x = data[inds[i]]['sol'].x

tT, R1T, R2T = Simulator_v3(td, td+tm+step, 100, True, Rd, Tjumps)
t, r1, r2 = Simulator_v3(x[0], x[0]+x[1], 100, True, [100, 100*x[2]], [0, x[0]*(1-x[3])])

R1T1 = np.interp(td+tm, tT, R1T)
R2T1 = np.interp(td+tm, tT, R2T)
result = inverseProblemSolver3_DoubleMeasurement(R1T1, R2T1, R1T[-1], R2T[-1], step)
print('\n\n\n\n')
print(result)

tdouble, r1double, r2double = Simulator_v3(result.x[0], result.x[0]+result.x[1]+step,
                                           100, True, [100, 100*result.x[2]], [0, result.x[0]*(1-result.x[3])])



{'td': 14523.677524838637, 'tm': 20.860698194480936, 'Rd': array([134.19465492, 107.26984726, 167.67603651,  39.41351721]), 'Tjumps': array([    0.        ,  4062.43534446,  7520.64267365, 11504.60378291]), 'sol':  final_simplex: (array([[2.08738292e+04, 1.55835119e+01, 2.25173441e-01, 1.40891452e-02],
       [2.08738292e+04, 1.55835119e+01, 2.25173441e-01, 1.40891452e-02],
       [2.08738292e+04, 1.55835119e+01, 2.25173441e-01, 1.40891452e-02],
       [2.08738291e+04, 1.55835120e+01, 2.25173442e-01, 1.40891453e-02],
       [2.08738291e+04, 1.55835120e+01, 2.25173443e-01, 1.40891454e-02]]), array([1.54551113e-11, 1.59447646e-10, 1.81402318e-10, 3.65738102e-10,
       4.45388881e-10]))
           fun: 1.5455111254763505e-11
       message: 'Optimization terminated successfully.'
          nfev: 596
           nit: 330
        status: 0
       success: True
             x: array([2.08738292e+04, 1.55835119e+01, 2.25173441e-01, 1.40891452e-02])}
0
2.4604871810378865e-06
1
2.95478065364575

In [10]:

plt.figure(figsize=(6, 6))

plt.subplot(211)
plt.plot((tT-td)[tT<=td+tm], R1T[tT<=td+tm], 'orange', label='Forward')
plt.plot(t-x[0], r1, 'b', label='Inverse - single measurement')
plt.plot([tm], [R1T1], 'o', color='orange')
plt.plot([t[-1]-x[0]], [r1[-1]], 'bo')
plt.grid()
plt.legend(prop={'weight': 'bold', 'size': 9})
#plt.xlabel('Time')
plt.ylabel('$r_1$', fontsize=18, fontweight='bold')

plt.xticks([-td, 0], ['$T_{birth}$', '$T_{death}$'], fontsize=16, fontweight='bold')
plt.yticks([], [])

plt.subplot(212)
plt.plot((tT-td)[tT<=td+tm], R2T[tT<=td+tm], 'orange', label='Forward')
plt.plot(t-x[0], r2, 'b', label='Inverse - single measurement')
plt.plot([tm], [R2T1], 'o', color='orange')
plt.plot([t[-1]-x[0]], [r2[-1]], 'bo')
plt.grid()
plt.legend(prop={'weight': 'bold', 'size': 9})
plt.xlabel('Time', fontsize=16, fontweight='bold')
plt.ylabel('$r_2$', fontsize=18, fontweight='bold')

plt.xticks([-td, 0], ['$T_{birth}$', '$T_{death}$'], fontsize=16, fontweight='bold')
plt.yticks([], [])

plt.tight_layout()



plt.figure(figsize=(6, 6))

plt.subplot(211)
plt.plot(tT-td, R1T, 'orange', label='Forward')
plt.plot(t-x[0], r1, 'b', label='Inverse - single measurement')
plt.plot(tdouble-result.x[0], r1double, color='purple', linestyle='-.', label='Inverse - double measurement')
plt.plot([tm, tm+step], [R1T1, R1T[-1]], 'o', color='orange')
plt.plot([t[-1]-x[0]], [r1[-1]], 'bo')
plt.plot([tdouble[-1]-result.x[0]-step, tdouble[-1]-result.x[0]],
         [np.interp(tdouble[-1]-result.x[0]-step, tdouble-result.x[0], r1double), r1double[-1]], 'x', color='purple')
plt.grid()
plt.legend(prop={'weight': 'bold', 'size': 9})
plt.xticks([-td, 0], ['$T_{birth}$', '$T_{death}$'], fontsize=16, fontweight='bold')
plt.yticks([], [])
#plt.xlabel('Time')
plt.ylabel('$r_1$', fontsize=18, fontweight='bold')

plt.subplot(212)
plt.plot(tT-td, R2T, 'orange', label='Forward')
plt.plot(t-x[0], r2, 'b', label='Inverse - single measurement')
plt.plot(tdouble-result.x[0], r2double, color='purple', linestyle='-.', label='Inverse - double measurement')
plt.plot([tm, tm+step], [R2T1, R2T[-1]], 'o', color='orange')
plt.plot([t[-1]-x[0]], [r2[-1]], 'bo')
plt.plot([tdouble[-1]-result.x[0]-step, tdouble[-1]-result.x[0]],
         [np.interp(tdouble[-1]-result.x[0]-step, tdouble-result.x[0], r2double), r2double[-1]], 'x', color='purple')
plt.grid()
plt.legend(prop={'weight': 'bold', 'size': 9})
plt.xticks([-td, 0], ['$T_{birth}$', '$T_{death}$'], fontsize=16, fontweight='bold')
plt.yticks([], [])
plt.xlabel('Time', fontsize=16, fontweight='bold')
plt.ylabel('$r_2$', fontsize=18, fontweight='bold')

plt.tight_layout()
plt.show()

NameError: name 'tdouble' is not defined